# Understanding IFC Files: Building Code Compliance Exercises

## What is IFC?

**IFC (Industry Foundation Classes)** is a standard data format for building information. It stores geometric, spatial, and semantic information about buildings in a structured way. Think of it as a database format for architectural and engineering data.

Industries use IFC to:
- Share building models across different software
- Extract specific information (spaces, materials, systems)
- Validate designs against building codes
- Run simulations and analysis

## Resources

Before starting, explore these resources to understand IFC better:

- **IFC Knowledge Base**: https://notebooklm.google.com/notebook/0925c2a1-519b-40a8-aca4-1e832d219f3c
- **BuildingSmart (IFC Standard)**: https://www.buildingsmart.org/standards/bsi-standards/industry-foundation-classes/
- **IfcOpenShell Python Docs**: https://docs.ifcopenshell.org/ifcopenshell-python.html
- **Catalan Building Code Reference**: https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e (for exercises 1 & bonus)

This notebook uses **IfcOpenShell**, a Python library for reading and writing IFC files.

## Setup: Load and Explore IFC Files

First, let's install IfcOpenShell and load the duplex apartment model.

In [24]:
# Install IfcOpenShell (run this once)
import subprocess
import sys

try:
    import ifcopenshell
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ifcopenshell", "-q"])
    import ifcopenshell

# Import other useful libraries
import json
from pathlib import Path
from collections import defaultdict

# Load the IFC file
ifc_file_path = Path("assets/duplex.ifc")
ifc = ifcopenshell.open(ifc_file_path)

print(f"✓ IFC file loaded: {ifc_file_path}")
print(f"✓ IFC Schema: {ifc.schema}")

✓ IFC file loaded: assets\duplex.ifc
✓ IFC Schema: IFC2X3


### What's Inside the File?

Let's explore the structure of the IFC model:

In [25]:
# Get all unique entity types in the model
all_entities = list(ifc)
entity_types = defaultdict(int)
for entity in all_entities:
    entity_types[entity.is_a()] += 1

# Show entity counts
print("Entity types in this model:")
print("-" * 40)
for entity_type in sorted(entity_types.keys()):
    count = entity_types[entity_type]
    print(f"  {entity_type}: {count}")

print(f"\nTotal entities: {len(all_entities)}")

# Find basic info
building = ifc.by_type("IfcBuilding")[0] if ifc.by_type("IfcBuilding") else None
if building:
    print(f"\nBuilding name: {building.Name}")
    print(f"Building description: {building.Description}")

Entity types in this model:
----------------------------------------
  IfcApplication: 1
  IfcArbitraryClosedProfileDef: 102
  IfcArbitraryOpenProfileDef: 184
  IfcArbitraryProfileDefWithVoids: 18
  IfcAxis2Placement2D: 397
  IfcAxis2Placement3D: 1279
  IfcBeam: 8
  IfcBooleanClippingResult: 7
  IfcBuilding: 1
  IfcBuildingStorey: 4
  IfcCartesianPoint: 8520
  IfcCartesianTransformationOperator3D: 167
  IfcCircle: 96
  IfcCircleProfileDef: 8
  IfcColourRgb: 43
  IfcCompositeCurve: 44
  IfcCompositeCurveSegment: 322
  IfcConnectedFaceSet: 235
  IfcConnectionSurfaceGeometry: 265
  IfcConversionBasedUnit: 1
  IfcCovering: 13
  IfcCurveBoundedPlane: 81
  IfcCurveStyle: 31
  IfcCurveStyleFont: 19
  IfcCurveStyleFontPattern: 57
  IfcDimensionalExponents: 1
  IfcDirection: 134
  IfcDoor: 14
  IfcDoorLiningProperties: 6
  IfcDoorStyle: 6
  IfcDraughtingPreDefinedCurveFont: 1
  IfcElementQuantity: 21
  IfcExtrudedAreaSolid: 421
  IfcFace: 4486
  IfcFaceBasedSurfaceModel: 40
  IfcFaceBound: 60
 

## Example: Extract and Display All IFC Spaces

An **IfcSpace** represents a room or area in the building. Each space has properties like name, area, and volume. Here's how to extract them:

In [26]:
# Get all spaces
spaces = ifc.by_type("IfcSpace")

print(f"Found {len(spaces)} spaces in the building\n")
print("=" * 60)

for i, space in enumerate(spaces, 1):
    # Extract properties
    name = space.Name if space.Name else "Unnamed"
    
    # Try to get area and volume
    area = None
    volume = None
    
    if hasattr(space, 'Quantities') and space.Quantities:
        for quantity in space.Quantities.Quantities:
            if hasattr(quantity, 'Name'):
                if quantity.Name == "NetFloorArea" and hasattr(quantity, 'AreaValue'):
                    area = quantity.AreaValue
                elif quantity.Name == "GrossVolume" and hasattr(quantity, 'VolumeValue'):
                    volume = quantity.VolumeValue
    
    # Format output
    print(f"{i}. {name}")
    if area:
        print(f"   Area: {area:.2f} m²")
    if volume:
        print(f"   Volume: {volume:.2f} m³")
    print()

print("=" * 60)

Found 21 spaces in the building

1. A101

2. A201

3. B104

4. B101

5. B201

6. A205

7. B205

8. A105

9. B105

10. R301

11. B102

12. A102

13. A103

14. B103

15. B204

16. A204

17. A104

18. B203

19. A202

20. B202

21. A203



## Exercise 1: Building Code Compliance Checker (Catalonia)

**Objective:** Write a function that validates spaces against Catalonia building code requirements.

**Reference:** [Catalan Building Code](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Key Requirements to Check

Based on the Catalan building code, residential spaces should meet these minimum standards:

| Space Type | Min Height | Min Area | Purpose |
|---|---|---|---|
| Living Room | 2.6 m | 16 m² | Main living area |
| Bedroom | 2.6 m | 9 m² | Single occupancy |
| Kitchen | 2.6 m | 8 m² | Cooking/dining |
| Bathroom | 2.3 m | 4 m² | Hygiene |
| Corridor | 2.3 m | 1.5 m² | Circulation |

### Your Task

Write a function `check_space_compliance(spaces)` that:

1. **Identifies** each space type (you can infer from the name or classify them)
2. **Extracts** the height and area properties from each space
3. **Compares** against the requirements above
4. **Reports** which spaces pass/fail and why
5. **Returns** a summary with compliance status

**Hints:**
- Height can be extracted from the Z-coordinate range of the space geometry
- Area is usually stored in space.Quantities as "NetFloorArea"
- Space names often indicate their type (e.g., "Master Bedroom", "Kitchen")

### Starter Code

In [27]:
# Exercise 1: Building Code Compliance Checker

def check_space_compliance(spaces):
    """
    Check each space for Catalan building code compliance.

    Args:
        spaces: List of IfcSpace objects

    Returns:
        dict: Compliance report with passed, failed, and warnings lists
    """

    requirements = {
        "Living Room": {"min_height": 2.6, "min_area": 16},
        "Bedroom":     {"min_height": 2.6, "min_area": 9},
        "Kitchen":     {"min_height": 2.6, "min_area": 8},
        "Bathroom":    {"min_height": 2.3, "min_area": 4},
        "Corridor":    {"min_height": 2.3, "min_area": 1.5},
    }

    # Keywords matched against LongName (which is descriptive in this file)
    type_keywords = {
        "Living Room": ["living", "salon", "lounge", "sala", "sitting", "dining"],
        "Bedroom":     ["bedroom", "bed", "dormitorio", "dorm", "master"],
        "Kitchen":     ["kitchen", "cocina", "cuina", "cook"],
        "Bathroom":    ["bathroom", "bath", "wc", "toilet", "lavabo", "bany", "aseo"],
        "Corridor":    ["corridor", "hall", "hallway", "foyer", "entry", "pasillo"],
    }

    report = {"passed": [], "failed": [], "warnings": []}

    for space in spaces:
        # ── Extract space properties ──────────────────────────────────────────
        name      = (space.Name     or "").strip()
        long_name = (space.LongName or "").strip() if space.LongName else ""

        area   = None
        height = None

        for rel in space.IsDefinedBy:
            prop_def = rel.RelatingPropertyDefinition

            # Area: the file stores it as "GSA BIM Area" (IfcQuantityArea)
            # → match by type, not by name, so any area quantity is captured
            if prop_def.is_a("IfcElementQuantity"):
                for qty in prop_def.Quantities:
                    if qty.is_a("IfcQuantityArea") and area is None:
                        area = qty.AreaValue

            # Height: stored in PSet_Revit_Dimensions as "Unbounded Height" property
            elif prop_def.is_a("IfcPropertySet") and prop_def.Name == "PSet_Revit_Dimensions":
                for prop in prop_def.HasProperties:
                    if prop.Name == "Unbounded Height":
                        height = prop.NominalValue.wrappedValue

        # ── Identify space type from LongName ─────────────────────────────────
        search_text = long_name.lower()
        space_type = next(
            (st for st, kws in type_keywords.items() if any(kw in search_text for kw in kws)),
            None
        )

        display_name = long_name if long_name else name
        space_info = {
            "id":         name,
            "name":       display_name,
            "space_type": space_type or "Unknown",
            "area_m2":    round(area,   2) if area   is not None else None,
            "height_m":   round(height, 2) if height is not None else None,
        }

        # ── Compare against requirements ──────────────────────────────────────
        if space_type is None:
            report["warnings"].append({
                **space_info,
                "reason": f"Cannot classify '{display_name}' – skipped"
            })
            continue

        req    = requirements[space_type]
        issues = []

        if area is None:
            issues.append("Area data not available")
        elif area < req["min_area"]:
            issues.append(f"Area {area:.2f} m² < required {req['min_area']} m²")

        if height is None:
            issues.append("Height data not available")
        elif height < req["min_height"]:
            issues.append(f"Height {height:.2f} m < required {req['min_height']} m")

        # ── Populate report ───────────────────────────────────────────────────
        if issues:
            report["failed"].append({**space_info, "issues": issues})
        else:
            report["passed"].append(space_info)

    return report


# ── Run & display results ─────────────────────────────────────────────────────
result = check_space_compliance(spaces)

print(f"Passed  : {len(result['passed'])} spaces")
print(f"Failed  : {len(result['failed'])} spaces")
print(f"Warnings: {len(result['warnings'])} spaces (unclassified)")

if result["passed"]:
    print("\n✓ PASSING SPACES")
    for s in result["passed"]:
        print(f"  [{s['id']}] {s['name']:<15} ({s['space_type']:<12})  "
              f"area={s['area_m2']:>6} m²  height={s['height_m']} m")

if result["failed"]:
    print("\n✗ FAILING SPACES")
    for s in result["failed"]:
        print(f"  [{s['id']}] {s['name']:<15} ({s['space_type']})")
        for issue in s["issues"]:
            print(f"      – {issue}")

if result["warnings"]:
    print("\n⚠ UNCLASSIFIED (no compliance check applied)")
    for s in result["warnings"]:
        print(f"  [{s['id']}] {s['name']:<15}  area={s['area_m2']} m²  height={s['height_m']} m")

Passed  : 14 spaces
Failed  : 2 spaces
Warnings: 5 spaces (unclassified)

✓ PASSING SPACES
  [A101] Foyer           (Corridor    )  area= 17.94 m²  height=2.6 m
  [A201] Hallway         (Corridor    )  area=   7.8 m²  height=2.9 m
  [B101] Foyer           (Corridor    )  area= 17.94 m²  height=2.6 m
  [B201] Hallway         (Corridor    )  area=   7.8 m²  height=2.9 m
  [B102] Living Room     (Living Room )  area= 30.14 m²  height=2.6 m
  [A102] Living Room     (Living Room )  area= 30.14 m²  height=2.6 m
  [A103] Kitchen         (Kitchen     )  area=  13.9 m²  height=2.6 m
  [B103] Kitchen         (Kitchen     )  area=  13.9 m²  height=2.6 m
  [B204] Bathroom 2      (Bathroom    )  area=  5.44 m²  height=2.6 m
  [A204] Bathroom 2      (Bathroom    )  area=  5.42 m²  height=2.6 m
  [B203] Bedroom 2       (Bedroom     )  area= 26.18 m²  height=2.6 m
  [A202] Bedroom 1       (Bedroom     )  area= 26.12 m²  height=2.6 m
  [B202] Bedroom 1       (Bedroom     )  area= 26.12 m²  height=2.6 m

In [28]:
# Display detailed compliance report
result = check_space_compliance(spaces)

print("=" * 70)
print("COMPLIANCE REPORT - CATALAN BUILDING CODE")
print("=" * 70)

print(f"\nSUMMARY:")
print(f"  Passed  : {len(result['passed'])} spaces")
print(f"  Failed  : {len(result['failed'])} spaces")
print(f"  Warnings: {len(result['warnings'])} spaces")

if result['passed']:
    print(f"\n{'PASSED SPACES:':-^70}")
    for space in result['passed']:
        print(f"\n  {space['name']} ({space['space_type']})")
        print(f"    Area: {space['area_m2']} m² | Height: {space['height_m']} m")

if result['failed']:
    print(f"\n{'FAILED SPACES (Non-Compliant):':-^70}")
    for space in result['failed']:
        print(f"\n  {space['name']} ({space['space_type']})")
        print(f"    Area: {space['area_m2']} m² | Height: {space['height_m']} m")
        for issue in space.get('issues', []):
            print(f"    ✗ {issue}")

if result['warnings']:
    print(f"\n{'WARNINGS (Unclassified):':-^70}")
    for warning in result['warnings']:
        print(f"\n  {warning['name']} ({warning['space_type']})")
        print(f"    ⚠ {warning['reason']}")

print("\n" + "=" * 70)

COMPLIANCE REPORT - CATALAN BUILDING CODE

SUMMARY:
  Passed  : 14 spaces
  Failed  : 2 spaces
  Warnings: 5 spaces

----------------------------PASSED SPACES:----------------------------

  Foyer (Corridor)
    Area: 17.94 m² | Height: 2.6 m

  Hallway (Corridor)
    Area: 7.8 m² | Height: 2.9 m

  Foyer (Corridor)
    Area: 17.94 m² | Height: 2.6 m

  Hallway (Corridor)
    Area: 7.8 m² | Height: 2.9 m

  Living Room (Living Room)
    Area: 30.14 m² | Height: 2.6 m

  Living Room (Living Room)
    Area: 30.14 m² | Height: 2.6 m

  Kitchen (Kitchen)
    Area: 13.9 m² | Height: 2.6 m

  Kitchen (Kitchen)
    Area: 13.9 m² | Height: 2.6 m

  Bathroom 2 (Bathroom)
    Area: 5.44 m² | Height: 2.6 m

  Bathroom 2 (Bathroom)
    Area: 5.42 m² | Height: 2.6 m

  Bedroom 2 (Bedroom)
    Area: 26.18 m² | Height: 2.6 m

  Bedroom 1 (Bedroom)
    Area: 26.12 m² | Height: 2.6 m

  Bedroom 1 (Bedroom)
    Area: 26.12 m² | Height: 2.6 m

  Bedroom 2 (Bedroom)
    Area: 26.18 m² | Height: 2.6 m

---

## Exercise 2: Window Detection and Compliance Verification

**Objective:** Find all windows in the model and verify they meet natural light and ventilation requirements.

**Reference:** [Catalan Building Code](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Key Requirements

Residential spaces must have:
- **Minimum window area** = 1/8 of floor area (12.5%)
- **Minimum window dimensions** = 60cm width, 100cm height (for single opening)
- Living areas should have direct natural light

### Your Task

Write a function `analyze_window_compliance(ifc_model, spaces)` that:

1. **Finds all IfcWindow** entities in the model
2. **Links windows to spaces** (which room is each window in?)
3. **Extracts properties**: dimensions, area, orientation
4. **Calculates window-to-floor ratio** for each space
5. **Reports compliance** for each space with windows

**Hints:**
- Windows are `IfcWindow` entities
- To link windows to spaces, check spatial containment relationships
- Window area can be calculated from the frame/pane dimensions
- You may need to examine the geometric representation

### Starter Code

In [29]:
# Exercise 2: Window Detection and Compliance Verification

def analyze_window_compliance(ifc_model, spaces):
    """
    Analyze windows in each space and check compliance.

    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects

    Returns:
        dict: Report with window analysis and compliance status
    """

    MIN_RATIO  = 1 / 8       # window area must be ≥ 12.5% of floor area
    MIN_WIDTH  = 0.60        # m
    MIN_HEIGHT = 1.00        # m

    windows = ifc_model.by_type("IfcWindow")

    report = {
        "total_windows": len(windows),
        "windows_by_space": {},
        "compliance_status": {}
    }

    print(f"Found {len(windows)} windows in the model\n")

    # ── Build a lookup: space GlobalId → space object ─────────────────────────
    space_by_id = {s.GlobalId: s for s in spaces}

    # ── Extract window properties ─────────────────────────────────────────────
    # OverallWidth / OverallHeight are direct IFC attributes on IfcWindow
    def get_window_info(win):
        w = win.OverallWidth  if win.OverallWidth  else 0.0
        h = win.OverallHeight if win.OverallHeight else 0.0
        return {"name": win.Name or "Unnamed", "width": w, "height": h, "area": w * h}

    # ── Group windows by space via IfcRelSpaceBoundary ────────────────────────
    # Each boundary record says: this building element borders this space
    windows_by_space = {}   # space GlobalId → list of window_info dicts

    for boundary in ifc_model.by_type("IfcRelSpaceBoundary"):
        elem = boundary.RelatedBuildingElement
        if elem is None or not elem.is_a("IfcWindow"):
            continue
        space = boundary.RelatingSpace
        if space.GlobalId not in space_by_id:
            continue
        sid = space.GlobalId
        if sid not in windows_by_space:
            windows_by_space[sid] = []
        # Avoid counting the same window twice for the same space
        info = get_window_info(elem)
        if not any(w["name"] == info["name"] for w in windows_by_space[sid]):
            windows_by_space[sid].append(info)

    report["windows_by_space"] = windows_by_space

    # ── Get floor area for each space (same method as Exercise 1) ────────────
    def get_floor_area(space):
        for rel in space.IsDefinedBy:
            prop_def = rel.RelatingPropertyDefinition
            if prop_def.is_a("IfcElementQuantity"):
                for qty in prop_def.Quantities:
                    if qty.is_a("IfcQuantityArea"):
                        return qty.AreaValue
        return None

    # ── Calculate window-to-floor ratio and check compliance ─────────────────
    print(f"{'Space':<8} {'Name':<15} {'Floor m²':>8} {'Win m²':>7} {'Ratio':>7}  Status")
    print("-" * 65)

    for space in spaces:
        sid        = space.GlobalId
        name       = space.Name or ""
        long_name  = space.LongName or name
        floor_area = get_floor_area(space)
        wins       = windows_by_space.get(sid, [])

        total_window_area = sum(w["area"] for w in wins)

        # Check window-to-floor ratio
        ratio = total_window_area / floor_area if floor_area else None

        issues = []
        if len(wins) == 0:
            issues.append("No windows found in this space")
        else:
            if ratio is not None and ratio < MIN_RATIO:
                issues.append(
                    f"Window/floor ratio {ratio:.1%} < required {MIN_RATIO:.1%}"
                )
            for w in wins:
                if w["width"] < MIN_WIDTH:
                    issues.append(
                        f"Window '{w['name']}' width {w['width']*100:.0f}cm < required 60cm"
                    )
                if w["height"] < MIN_HEIGHT:
                    issues.append(
                        f"Window '{w['name']}' height {w['height']*100:.0f}cm < required 100cm"
                    )

        compliant = len(issues) == 0 and len(wins) > 0
        status    = "✓ PASS" if compliant else ("⚠ NO WIN" if not wins else "✗ FAIL")

        floor_str  = f"{floor_area:.2f}" if floor_area else "N/A"
        win_str    = f"{total_window_area:.2f}" if wins else "0"
        ratio_str  = f"{ratio:.1%}" if ratio is not None else "N/A"

        print(f"{name:<8} {long_name:<15} {floor_str:>8} {win_str:>7} {ratio_str:>7}  {status}")

        report["compliance_status"][sid] = {
            "name": long_name, "floor_area_m2": floor_area,
            "total_window_area_m2": total_window_area,
            "window_to_floor_ratio": ratio, "windows": wins,
            "compliant": compliant, "issues": issues
        }

    # ── Summary ───────────────────────────────────────────────────────────────
    statuses  = list(report["compliance_status"].values())
    passed    = [s for s in statuses if s["compliant"]]
    failed    = [s for s in statuses if not s["compliant"] and s["windows"]]
    no_windows= [s for s in statuses if not s["windows"]]

    print(f"\nPassed     : {len(passed)} spaces")
    print(f"Failed     : {len(failed)} spaces")
    print(f"No windows : {len(no_windows)} spaces")

    if failed:
        print("\nIssues found:")
        for s in failed:
            print(f"  [{s['name']}]")
            for issue in s["issues"]:
                print(f"    – {issue}")

    return report


# ── Run ───────────────────────────────────────────────────────────────────────
analysis = analyze_window_compliance(ifc, spaces)

Found 24 windows in the model

Space    Name            Floor m²  Win m²   Ratio  Status
-----------------------------------------------------------------
A101     Foyer              17.94       0    0.0%  ⚠ NO WIN
A201     Hallway             7.80       0    0.0%  ⚠ NO WIN
B104     Bathroom 1          4.00       0    0.0%  ⚠ NO WIN
B101     Foyer              17.94       0    0.0%  ⚠ NO WIN
B201     Hallway             7.80       0    0.0%  ⚠ NO WIN
A205     Utility             1.75       0    0.0%  ⚠ NO WIN
B205     Utility             1.73       0    0.0%  ⚠ NO WIN
A105     Stair               4.92       0    0.0%  ⚠ NO WIN
B105     Room                4.92       0    0.0%  ⚠ NO WIN
R301     Roof              145.72       0    0.0%  ⚠ NO WIN
B102     Living Room        30.14   13.35   44.3%  ✓ PASS
A102     Living Room        30.14   13.35   44.3%  ✓ PASS
A103     Kitchen            13.90    1.65   11.9%  ✗ FAIL
B103     Kitchen            13.90    1.65   11.9%  ✗ FAIL
B204     Bath

## Bonus Exercise: Fire Safety Route Analysis

**Objective:** Find the longest evacuation route within the apartment and verify it meets fire safety requirements.

**Difficulty:** Advanced

**Reference:** [Catalan Building Code - Fire Safety Section](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Fire Safety Requirements

According to the Catalan building code:
- **Maximum travel distance** to exit: ≤ 25-30 m (depending on building type)
- **Minimum corridor width**: 1.2 m
- **Minimum door width**: 0.8 m (for exits)
- **No dead-end corridors** longer than 10 m

### Your Task

Write a function `analyze_evacuation_routes(ifc_model, spaces)` that:

1. **Builds a spatial graph** of the apartment (rooms as nodes, doors/openings as connections)
2. **Calculates distances** between spaces (using area/perimeter as proxy)
3. **Finds the longest route** from any point to the nearest exit
4. **Validates** the route against fire safety requirements
5. **Identifies bottlenecks** (narrow corridors, small doors)

**Hints:**
- Think of spaces as nodes and doors as edges in a graph
- Use BFS/DFS to find longest paths
- Door dimensions can indicate width constraints
- Consider calculating distances based on space geometry
- This is a simplified model - real analysis would use detailed geometry

### Starter Code

```python
def analyze_evacuation_routes(ifc_model, spaces):
    """
    Analyze evacuation routes and fire safety compliance.
    
    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Fire safety analysis and recommendations
    """
    
    # Get all doors (potential connections between spaces)
    doors = ifc_model.by_type("IfcDoor")
    
    analysis = {
        "total_spaces": len(spaces),
        "total_doors": len(doors),
        "longest_route": None,
        "longest_distance": 0,
        "safety_issues": [],
        "compliant": False
    }
    
    print(f"Analyzing {len(spaces)} spaces with {len(doors)} doors")
    
    # TODO: Build spatial connectivity graph
    # TODO: Find longest evacuation path
    # TODO: Check corridor widths and door dimensions
    # TODO: Validate against requirements
    # TODO: Report issues and recommendations
    
    return analysis


# Run the analysis (uncomment when ready)
# fire_analysis = analyze_evacuation_routes(ifc, spaces)
```

In [30]:
# Bonus Exercise: Fire Safety Route Analysis

from collections import defaultdict, deque
import math

def analyze_evacuation_routes(ifc_model, spaces):
    """
    Analyze evacuation routes and fire safety compliance.

    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects

    Returns:
        dict: Fire safety analysis and recommendations
    """

    MAX_TRAVEL_DISTANCE = 25.0   # m — Catalan code max travel distance to exit
    MIN_DOOR_WIDTH      = 0.80   # m — minimum exit door width
    MIN_CORRIDOR_WIDTH  = 1.20   # m — minimum corridor width (checked via door widths)

    doors = ifc_model.by_type("IfcDoor")

    analysis = {
        "total_spaces": len(spaces),
        "total_doors":  len(doors),
        "longest_route": None,
        "longest_distance": 0,
        "safety_issues": [],
        "compliant": False
    }

    print(f"Analyzing {len(spaces)} spaces with {len(doors)} doors\n")

    # ── Helper: get floor area from quantities ────────────────────────────────
    def get_area(space):
        for rel in space.IsDefinedBy:
            pd = rel.RelatingPropertyDefinition
            if pd.is_a("IfcElementQuantity"):
                for qty in pd.Quantities:
                    if qty.is_a("IfcQuantityArea"):
                        return qty.AreaValue
        return 1.0   # fallback

    space_by_name = {s.Name: s for s in spaces}

    # ── Build spatial connectivity graph via IfcRelSpaceBoundary ─────────────
    # A door touching exactly one space = exterior door (potential exit)
    # A door touching two spaces        = internal door (graph edge)

    door_to_spaces = defaultdict(set)   # door GlobalId → {space Name, ...}
    door_width     = {}                 # door GlobalId → width (m)

    for boundary in ifc_model.by_type("IfcRelSpaceBoundary"):
        elem = boundary.RelatedBuildingElement
        if elem is None or not elem.is_a("IfcDoor"):
            continue
        door_to_spaces[elem.GlobalId].add(boundary.RelatingSpace.Name)
        door_width[elem.GlobalId] = elem.OverallWidth or 0.0

    # Graph: space Name → list of (neighbour Name, door width)
    graph   = defaultdict(list)
    # Exits: spaces that have an exterior door
    exits   = set()

    for gid, space_names in door_to_spaces.items():
        names = list(space_names)
        w     = door_width[gid]

        if len(names) == 1:
            # Exterior door — this space is an exit node
            exits.add(names[0])
        elif len(names) == 2:
            # Internal door — bidirectional edge
            graph[names[0]].append((names[1], w))
            graph[names[1]].append((names[0], w))

    print(f"Exit spaces (have exterior door): {sorted(exits)}")
    print(f"Internal door connections: {sum(len(v) for v in graph.values()) // 2}\n")

    # ── Dijkstra from ALL exits simultaneously (multi-source) ─────────────────
    # Distance proxy: sqrt(area) of each room represents traversal distance through it.
    # We want the SHORTEST path to any exit for every space, then report the WORST one.

    INF  = float("inf")
    dist = {s.Name: INF for s in spaces}
    prev = {s.Name: None for s in spaces}

    # Priority queue seeded with all exits at distance 0
    import heapq
    pq = []
    for exit_name in exits:
        dist[exit_name] = 0.0
        heapq.heappush(pq, (0.0, exit_name))

    while pq:
        d, u = heapq.heappop(pq)
        if d > dist[u]:
            continue
        for v, door_w in graph[u]:
            # Cost to traverse space v = sqrt of its floor area (diameter proxy)
            space_v   = space_by_name.get(v)
            step_cost = math.sqrt(get_area(space_v)) if space_v else 1.0
            new_dist  = d + step_cost
            if new_dist < dist[v]:
                dist[v] = new_dist
                prev[v] = u
                heapq.heappush(pq, (new_dist, v))

    # ── Find the worst-case (longest shortest path) ───────────────────────────
    reachable   = {n: d for n, d in dist.items() if d < INF}
    unreachable = [n for n, d in dist.items() if d == INF]

    if reachable:
        worst_space = max(reachable, key=reachable.get)
        worst_dist  = reachable[worst_space]

        # Reconstruct route
        route = []
        node  = worst_space
        while node is not None:
            route.append(node)
            node = prev[node]
        route_names = " → ".join(
            f"{n}({space_by_name[n].LongName or n})" if n in space_by_name else n
            for n in route
        )

        analysis["longest_route"]    = route_names
        analysis["longest_distance"] = round(worst_dist, 2)

    # ── Check door widths ─────────────────────────────────────────────────────
    narrow_doors = []
    for gid, w in door_width.items():
        if w < MIN_DOOR_WIDTH:
            door_obj   = ifc_model.by_guid(gid)
            space_list = ", ".join(door_to_spaces[gid])
            narrow_doors.append(
                f"Door '{door_obj.Name}' ({w*100:.0f}cm wide) "
                f"in spaces [{space_list}] — min {MIN_DOOR_WIDTH*100:.0f}cm required"
            )

    # ── Collect safety issues ─────────────────────────────────────────────────
    if worst_dist > MAX_TRAVEL_DISTANCE:
        analysis["safety_issues"].append(
            f"Longest evacuation route {worst_dist:.1f}m exceeds {MAX_TRAVEL_DISTANCE}m limit"
        )
    for msg in narrow_doors:
        analysis["safety_issues"].append(msg)
    for name in unreachable:
        s = space_by_name.get(name)
        ln = s.LongName if s else name
        analysis["safety_issues"].append(f"Space {name} ({ln}) has NO path to any exit")

    analysis["compliant"] = len(analysis["safety_issues"]) == 0

    # ── Print report ──────────────────────────────────────────────────────────
    print("EVACUATION DISTANCE TO NEAREST EXIT")
    print(f"{'Space':<8} {'Name':<15} {'Distance (m)':>13}  {'Status'}")
    print("-" * 50)
    for space in spaces:
        n  = space.Name
        ln = space.LongName or n
        d  = dist.get(n, INF)
        if d == INF:
            status = "✗ UNREACHABLE"
        elif d == 0:
            status = "★ EXIT"
        elif d <= MAX_TRAVEL_DISTANCE:
            status = "✓ OK"
        else:
            status = "✗ TOO FAR"
        d_str = f"{d:.1f}" if d < INF else "∞"
        print(f"{n:<8} {ln:<15} {d_str:>13}  {status}")

    print(f"\nWorst-case route : {analysis['longest_route']}")
    print(f"Worst distance   : {analysis['longest_distance']} m  (limit: {MAX_TRAVEL_DISTANCE} m)")

    if analysis["compliant"]:
        print("\n✓ All evacuation routes are compliant")
    else:
        print("\n✗ Safety issues found:")
        for issue in analysis["safety_issues"]:
            print(f"  – {issue}")

    return analysis


# ── Run ───────────────────────────────────────────────────────────────────────
fire_analysis = analyze_evacuation_routes(ifc, spaces)


Analyzing 21 spaces with 14 doors

Exit spaces (have exterior door): ['A101', 'A102', 'B101', 'B102']
Internal door connections: 9

EVACUATION DISTANCE TO NEAREST EXIT
Space    Name             Distance (m)  Status
--------------------------------------------------
A101     Foyer                     0.0  ★ EXIT
A201     Hallway                     ∞  ✗ UNREACHABLE
B104     Bathroom 1                2.0  ✓ OK
B101     Foyer                     0.0  ★ EXIT
B201     Hallway                     ∞  ✗ UNREACHABLE
A205     Utility                     ∞  ✗ UNREACHABLE
B205     Utility                     ∞  ✗ UNREACHABLE
A105     Stair                       ∞  ✗ UNREACHABLE
B105     Room                        ∞  ✗ UNREACHABLE
R301     Roof                        ∞  ✗ UNREACHABLE
B102     Living Room               0.0  ★ EXIT
A102     Living Room               0.0  ★ EXIT
A103     Kitchen                     ∞  ✗ UNREACHABLE
B103     Kitchen                     ∞  ✗ UNREACHABLE
B204     Bathro

## Useful IFC Concepts

### Common Entity Types

- **IfcSpace**: A room, area, or zone in the building
- **IfcWindow**: Windows for light and ventilation  
- **IfcDoor**: Doors, openings, access points
- **IfcWall**: Boundary elements
- **IfcBuildingElement**: General building components
- **IfcElement**: Physical building pieces with properties

### Accessing Properties

```python
# Get all entities of a type
elements = ifc.by_type("IfcWindow")

# Access properties
entity.Name                    # String name
entity.Description             # Text description
entity.ObjectPlacement        # Location/coordinates
entity.Representation         # Geometry data
entity.QuantityInSpace         # Calculate-derived quantities

# Common relationships
entity.HasProperties           # Get properties
entity.BoundedBy               # Spatial boundaries
entity.HostedBy                # Connection relationships
```

### Helpful Methods

```python
# Query by GUID
entity = ifc.by_id(guid)

# Filter by property value
elements = ifc.by_attribute("Name", "Kitchen")

# Get all instances of a type
spaces = ifc.by_type("IfcSpace")

# Check entity type
if space.is_a("IfcSpace"):
    print("This is a space")
```

## Resources for Help

- **IFC Knowledge Base**: https://notebooklm.google.com/notebook/0925c2a1-519b-40a8-aca4-1e832d219f3c
- **IfcOpenShell Documentation**: https://docs.ifcopenshell.org/ifcopenshell-python.html
- **BuildingSmart Standards**: https://www.buildingsmart.org/
- **Catalan Building Code**: https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e

## Next Steps

After completing these exercises, you'll have learned:
- ✓ How to load and explore IFC files
- ✓ How to extract spatial and building data
- ✓ How to validate designs against building codes
- ✓ How to work with doors, windows, and routes

These skills apply to real-world AEC (Architecture, Engineering, Construction) workflows.